In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tweet-sentiment-extraction/sample_submission.csv
/kaggle/input/tweet-sentiment-extraction/train.csv
/kaggle/input/tweet-sentiment-extraction/test.csv


In [3]:
star = pd.read_csv('https://github.com/YBIFoundation/Dataset/raw/main/Stars.csv')

In [6]:
import pandas as pd
import numpy as np
import time
star = pd.read_csv ('https://github.com/YBIFoundation/Dataset/raw/main/Stars.csv')
star.replace({'Spectral Class':{'M':0, 'A':1, 'B':1, 'F':1, 'O':1, 'K':1, 'G':1 }}, inplace=True)
star.replace({'Star color':{ 'Red':0, 'Yellow':1, 'White':2, 'White ': 2, 'Blue ':3, 'Blue':3 }}, inplace=True)
y = star['Spectral Class']
X = star[['Temperature (K)', 'Luminosity (L/Lo)', 'Radius (R/Ro)',
       'Absolute magnitude (Mv)']]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state = 200)

from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
start = time.time()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
end = time.time()
temps_mis=end-start
y_pred
from sklearn.metrics import confusion_matrix, classification_report
confusion_matrix(y_test,y_pred)
print(classification_report(y_test,y_pred))
print(temps_mis)

              precision    recall  f1-score   support

           0       0.93      0.78      0.85        18
           1       0.88      0.97      0.92        30

    accuracy                           0.90        48
   macro avg       0.91      0.87      0.88        48
weighted avg       0.90      0.90      0.89        48

0.012314319610595703


In [8]:
import os
import spacy
import pandas as pd
from spacy.util import minibatch, compounding
import random
from tqdm import tqdm
import time

# Charger les données
df_train = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/train.csv')
df_test = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/test.csv')
df_submission = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/sample_submission.csv')

# Nettoyage des données
df_train['Num_words_text'] = df_train['text'].apply(lambda x: len(str(x).split()))
df_train = df_train[df_train['Num_words_text'] >= 3]

def save_model(output_dir, nlp, new_model_name):
    output_dir = f'/kaggle/working/{output_dir}'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    nlp.meta["name"] = new_model_name
    nlp.to_disk(output_dir)
    print(f"Saved model to {output_dir}")


from spacy.training import Example

def train(train_data, output_dir, n_iter=20, model=None):
    start_time = time.time()
    if model:
        nlp = spacy.load(output_dir)
        print(f"Loaded model '{model}'")
    else:
        nlp = spacy.blank("en")
        print("Created blank 'en' model")

    # Ajout ou récupération du composant NER
    if "ner" not in nlp.pipe_names:
        nlp.add_pipe("ner", last=True)
    ner = nlp.get_pipe("ner")

    # Ajouter les labels
    for _, annotations in train_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):
        if model is None:
            nlp.begin_training()
        else:
            nlp.resume_training()

        for itn in tqdm(range(n_iter)):
            random.shuffle(train_data)
            batches = minibatch(train_data, size=compounding(4.0, 500.0, 1.001))
            losses = {}
            for batch in batches:
                examples = [Example.from_dict(nlp.make_doc(text), annotations) for text, annotations in batch]  # ✅ Utilisation de Example.from_dict
                nlp.update(examples, drop=0.5, losses=losses)
            print("Losses", losses)

    save_model(output_dir, nlp, 'st_ner')
    end_time = time.time()
    print(f"Training completed in {end_time - start_time:.2f} seconds")

   


def get_model_out_path(sentiment):
    if sentiment == 'positive':
        return 'models/model_pos'
    elif sentiment == 'negative':
        return 'models/model_neg'
    return None

def get_training_data(sentiment):
    train_data = []
    for _, row in df_train.iterrows():
        if row.sentiment == sentiment:
            selected_text = row.selected_text
            text = row.text
            start = text.find(selected_text)
            end = start + len(selected_text)
            train_data.append((text, {"entities": [[start, end, 'selected_text']]}))
    return train_data

# Entraînement pour les sentiments positifs
for sentiment in ['positive', 'negative']:
    train_data = get_training_data(sentiment)
    model_path = get_model_out_path(sentiment)
    train(train_data, model_path, n_iter=3)

# Prédiction des entités
def predict_entities(text, model):
    doc = model(text)
    ent_array = []
    for ent in doc.ents:
        start = text.find(ent.text)
        end = start + len(ent.text)
        ent_array.append([start, end, ent.label_])
    return text[ent_array[0][0]: ent_array[0][1]] if ent_array else text

selected_texts = []
MODELS_BASE_PATH = '/kaggle/working/models/'

# Charger les modèles et faire des prédictions
model_pos = spacy.load(MODELS_BASE_PATH + 'model_pos')
model_neg = spacy.load(MODELS_BASE_PATH + 'model_neg')

for _, row in df_test.iterrows():
    text = row.text
    if row.sentiment == 'neutral' or len(text.split()) <= 2:
        selected_texts.append(text)
    elif row.sentiment == 'positive':
        selected_texts.append(predict_entities(text, model_pos))
    else:
        selected_texts.append(predict_entities(text, model_neg))

df_test['selected_text'] = selected_texts
df_submission['selected_text'] = df_test['selected_text']
df_submission.to_csv("/kaggle/working/submission.csv", index=False)
display(df_submission.head(10))


[2025-01-05 14:42:55,478] [INFO] Created vocabulary
[2025-01-05 14:42:55,479] [INFO] Finished initializing nlp object


Created blank 'en' model


  0%|          | 0/3 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Had a nice concert yesterday. Nice location, nice ..." with entities "[[6, 13, 'selected_text']]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text " lol. i get about the same reaction when i mention..." with entities "[[82, 113, 'selected_text']]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the 

Losses {'ner': 12823.673186150727}


 67%|██████▋   | 2/3 [01:38<00:49, 49.29s/it]

...

 67%|██████▋   | 2/3 [01:32<00:46, 46.06s/it]

Losses {'ner': 11418.131059615487}


100%|██████████| 3/3 [02:18<00:00, 46.03s/it]

Losses {'ner': 11139.730438496197}
Saved model to /kaggle/working/models/model_neg
Training completed in 138.47 seconds


,textID,selected_text
0,f87dea47db,Last session of the day http://twitpic.com/67ezh
1,96d74cb729,Good
2,eee518ae67,"Recession hit Veronique Branquinho, she has to..."
3,01082688c6,happy bday!
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!
5,726e501993,that`s great!
6,261932614e,I THINK EVERYONE HATES ME ON HERE lol
7,afa11da83f,"soooooo wish i could, but im in school and my..."
8,e64208b4ef,and within a short time of the last clue all ...
9,37bcad24ca,What did you get? My day is alright.. haven`...


In [10]:
import spacy

def load_model_and_test(model_path, test_texts):
    # Charger le modèle sauvegardé
    nlp = spacy.load(model_path)
    print(f"Model loaded from: {model_path}")
    
    # Tester sur des exemples
    for text in test_texts:
        doc = nlp(text)
        print(f"\nTexte: {text}")
        print("Entités détectées:")
        for ent in doc.ents:
            print(f" - {ent.text} ({ent.label_})")
        if not doc.ents:
            print("Aucune entité détectée.")
            
# Exemple d'utilisation
test_texts = [
    "I love this movie, it was amazing!",
    "The food was terrible, I hated it.",
    "I'm feeling neutral about this situation."
]

# Charger et tester le modèle positif
load_model_and_test('/kaggle/working/models/model_pos', test_texts)




Model loaded from: /kaggle/working/models/model_pos

Texte: I love this movie, it was amazing!
Entités détectées:
 - I love (selected_text)

Texte: The food was terrible, I hated it.
Entités détectées:
Aucune entité détectée.

Texte: I'm feeling neutral about this situation.
Entités détectées:
Aucune entité détectée.
